In [1]:
import os, pickle, h5py
os.chdir('/home/shimingwang/workspace/sf_tv/sceneflow_tv_se')

In [2]:
data_dir = 'data/argoverse2_demo/sensor/val'

## Process the data and visualize with our tool

In [3]:
data_index = pickle.load(open(os.path.join(data_dir, 'index_total.pkl'), 'rb'))

In [4]:
print(len(data_index))
print(data_index) ## scene_id, timestamp

157
[['25e5c600-36fe-3245-9cc0-40ef91620c22', '315966104259967000'], ['25e5c600-36fe-3245-9cc0-40ef91620c22', '315966104360163000'], ['25e5c600-36fe-3245-9cc0-40ef91620c22', '315966104459696000'], ['25e5c600-36fe-3245-9cc0-40ef91620c22', '315966104559893000'], ['25e5c600-36fe-3245-9cc0-40ef91620c22', '315966104660089000'], ['25e5c600-36fe-3245-9cc0-40ef91620c22', '315966104760285000'], ['25e5c600-36fe-3245-9cc0-40ef91620c22', '315966104859818000'], ['25e5c600-36fe-3245-9cc0-40ef91620c22', '315966104960015000'], ['25e5c600-36fe-3245-9cc0-40ef91620c22', '315966105060206000'], ['25e5c600-36fe-3245-9cc0-40ef91620c22', '315966105159739000'], ['25e5c600-36fe-3245-9cc0-40ef91620c22', '315966105259935000'], ['25e5c600-36fe-3245-9cc0-40ef91620c22', '315966105360132000'], ['25e5c600-36fe-3245-9cc0-40ef91620c22', '315966105460328000'], ['25e5c600-36fe-3245-9cc0-40ef91620c22', '315966105559861000'], ['25e5c600-36fe-3245-9cc0-40ef91620c22', '315966105660058000'], ['25e5c600-36fe-3245-9cc0-40ef91620

In [7]:
index = 32
scene_id, timestamp = data_index[index]
key = str(timestamp)
vis_name = 'seflow_best'
save_dir = 'outputs/reformulated_vis_outputs'
pkl_file_path = os.path.join(save_dir, scene_id, f'{timestamp}.pkl')

data_dict = pickle.load(open(pkl_file_path, 'rb'))

print(data_dict.keys())

dict_keys(['scene_id', 'timestamp', 'pc0', 'gm0', 'pose0', 'seflow_best', 'pose1', 'pc1', 'gm1'])


In [9]:
import numpy as np

In [13]:
pc0 = data_dict['pc0']
gm0 = data_dict['gm0']
pc1 = data_dict['pc1']
gm1 = data_dict['gm1']

pose0 = data_dict['pose0'] # ego2city
pose1 = data_dict['pose1'] # ego2city

ego0t1 = np.linalg.inv(pose1) @ pose0

pose_flow = pc0[:, :3] @ ego0t1[:3, :3].T + ego0t1[:3, 3] - pc0[:, :3] ## transform pc0 to pc0 after ego motion compensation

flow = data_dict['seflow_best']

true_flow = flow - pose_flow
print(f'pc0: {pc0.shape}')
print(f'gm0: {gm0.shape}')
print(f'pc1: {pc1.shape}')
print(f'gm1: {gm1.shape}')
print(f'pose0: {pose0.shape}')
print(f'pose1: {pose1.shape}')
print(f'ego0t1: {ego0t1.shape}')
print(f'pose_flow: {pose_flow.shape}')
print(f'seflow:{flow.shape}')
print(f'true_flow:{true_flow.shape}')

pc0: (95839, 3)
gm0: (95839,)
pc1: (95821, 3)
gm1: (95821,)
pose0: (4, 4)
pose1: (4, 4)
ego0t1: (4, 4)
pose_flow: (95839, 3)
seflow:(95839, 3)


## Investigate the data in hdf5 file

In [10]:
index = 32

In [16]:
scene_id, timestamp = data_index[index]
key = str(timestamp)
data_dict = {
    'scene_id': scene_id,
    'timestamp': timestamp,
}

vis_name = 'seflow_best'

In [17]:
with h5py.File(os.path.join(data_dir, f'{scene_id}.h5'), 'r') as f:
    data_dict['pc0'] = f[key]['lidar'][:]
    data_dict['gm0'] = f[key]['ground_mask'][:]
    data_dict['pose0'] = f[key]['pose'][:]
    for flow_key in [vis_name, 'dufo_label', 'label']:
        if flow_key in f[key]:
            data_dict[flow_key] = f[key][flow_key][:]

    next_timestamp = str(data_index[index+1][1])
    data_dict['pose1'] = f[next_timestamp]['pose'][:]
    data_dict['pc1'] = f[next_timestamp]['lidar'][:]
    data_dict['gm1'] = f[next_timestamp]['ground_mask'][:]

## Assemable

In [26]:
from tqdm import tqdm

In [27]:
save_dir = 'outputs/reformulated_vis_outputs'
for (scene_id, timestamp) in data_index:
    key = str(timestamp)
    data_dict = {
        'scene_id': scene_id,
        'timestamp': timestamp,
    }

    vis_name = 'seflow_best'
    with h5py.File(os.path.join(data_dir, f'{scene_id}.h5'), 'r') as f:
        data_dict['pc0'] = f[key]['lidar'][:]
        data_dict['gm0'] = f[key]['ground_mask'][:]
        data_dict['pose0'] = f[key]['pose'][:]
        for flow_key in [vis_name, 'dufo_label', 'label']:
            if flow_key in f[key]:
                data_dict[flow_key] = f[key][flow_key][:]

        next_timestamp = str(data_index[index+1][1])
        data_dict['pose1'] = f[next_timestamp]['pose'][:]
        data_dict['pc1'] = f[next_timestamp]['lidar'][:]
        data_dict['gm1'] = f[next_timestamp]['ground_mask'][:]
    
    sub_dir = os.path.join(save_dir,f'{scene_id}')
    if not os.path.exists(sub_dir):
        os.makedirs(sub_dir)
    file = os.path.join(sub_dir,f'{timestamp}.pkl')
    pickle.dump(data_dict, open(file, 'wb'))